# Chapter 10-3: 최종 프로젝트 - 종합 퀴즈 게임 (완성본)

## 🎯 프로젝트 개요
지금까지 배운 모든 Python 지식을 종합한 **완전한 퀴즈 게임**입니다!

### 포함된 기능
- 객체지향 설계 및 예외 처리
- 파일 입출력 및 JSON 데이터 관리
- 사용자 인터페이스 및 게임 진행
- 점수 시스템 및 통계 관리

In [ ]:
# 필요한 모듈들 import
import json
import random
import time
import os
from datetime import datetime
from typing import List, Dict, Any, Optional

print("📚 모든 필요한 모듈이 로드되었습니다.")

In [ ]:
# === 사용자 정의 예외 클래스들 ===

class QuizGameError(Exception):
    """퀴즈 게임 기본 예외"""
    pass

class InvalidPlayerNameError(QuizGameError):
    """잘못된 플레이어 이름"""
    pass

class InvalidQuestionError(QuizGameError):
    """잘못된 문제 형식"""
    pass

class GameError(QuizGameError):
    """게임 진행 관련 예외"""
    pass

print("✅ 예외 클래스들이 정의되었습니다.")

In [ ]:
# === Question 클래스 ===

class Question:
    """퀴즈 문제 클래스"""
    
    def __init__(self, text, options, correct, category, difficulty, explanation=""):
        self.text = text
        self.options = options
        self.correct = correct
        self.category = category
        self.difficulty = difficulty
        self.explanation = explanation
        self._validate()
    
    def _validate(self):
        if not self.text.strip():
            raise InvalidQuestionError("문제 텍스트가 비어있습니다.")
        if len(self.options) < 2:
            raise InvalidQuestionError("선택지가 최소 2개 필요합니다.")
        if not (0 <= self.correct < len(self.options)):
            raise InvalidQuestionError("정답 번호가 유효하지 않습니다.")
    
    def get_points(self):
        points = {'easy': 10, 'medium': 20, 'hard': 30}
        return points.get(self.difficulty, 10)
    
    def to_dict(self):
        return {
            'text': self.text,
            'options': self.options,
            'correct': self.correct,
            'category': self.category,
            'difficulty': self.difficulty,
            'explanation': self.explanation
        }
    
    @classmethod
    def from_dict(cls, data):
        return cls(**data)

print("✅ Question 클래스가 정의되었습니다.")

In [ ]:
# === Player 클래스 ===

class Player:
    """플레이어 클래스"""
    
    def __init__(self, name):
        self.name = self._validate_name(name)
        self.total_score = 0
        self.total_games = 0
        self.game_history = []
    
    def _validate_name(self, name):
        if not isinstance(name, str) or not name.strip():
            raise InvalidPlayerNameError("유효하지 않은 이름입니다.")
        name = name.strip()
        if len(name) < 2 or len(name) > 20:
            raise InvalidPlayerNameError("이름은 2-20글자여야 합니다.")
        return name
    
    def add_game_result(self, score, correct, total, category):
        result = {
            'date': datetime.now().isoformat(),
            'score': score,
            'correct': correct,
            'total': total,
            'category': category,
            'accuracy': (correct / total * 100) if total > 0 else 0
        }
        self.game_history.append(result)
        self.total_score += score
        self.total_games += 1
    
    def get_accuracy(self):
        if not self.game_history:
            return 0
        total_correct = sum(g['correct'] for g in self.game_history)
        total_questions = sum(g['total'] for g in self.game_history)
        return (total_correct / total_questions * 100) if total_questions > 0 else 0
    
    def get_best_score(self):
        return max((g['score'] for g in self.game_history), default=0)
    
    def to_dict(self):
        return {
            'name': self.name,
            'total_score': self.total_score,
            'total_games': self.total_games,
            'game_history': self.game_history
        }
    
    @classmethod
    def from_dict(cls, data):
        player = cls(data['name'])
        player.total_score = data.get('total_score', 0)
        player.total_games = data.get('total_games', 0)
        player.game_history = data.get('game_history', [])
        return player

print("✅ Player 클래스가 정의되었습니다.")

In [ ]:
# === QuizGame 클래스 (메인 게임 관리자) ===

class QuizGame:
    """퀴즈 게임 메인 클래스"""
    
    def __init__(self):
        self.questions = []
        self.players = {}
        self.current_player = None
        self.current_game = None
        self.data_dir = 'quiz_data'
        self._ensure_data_dir()
        self._load_default_questions()
        self._load_players()
    
    def _ensure_data_dir(self):
        """데이터 디렉토리 생성"""
        os.makedirs(self.data_dir, exist_ok=True)
    
    def _load_default_questions(self):
        """기본 문제들 로드"""
        default_questions = [
            Question("Python에서 리스트를 정의하는 방법은?", 
                    ["()", "[]", "{}", "''"], 1, "python", "easy", "대괄호 사용"),
            Question("Python의 불변 자료형은?", 
                    ["list", "dict", "tuple", "set"], 2, "python", "medium", "tuple은 불변"),
            Question("2의 8제곱은?", 
                    ["128", "256", "512", "64"], 1, "math", "easy", "2^8 = 256"),
            Question("대한민국의 수도는?", 
                    ["부산", "인천", "서울", "대구"], 2, "general", "easy", "서울이 수도"),
            Question("파이썬에서 예외처리 키워드는?", 
                    ["catch", "try", "handle", "error"], 1, "python", "medium", "try-except 사용"),
            Question("9 x 7 = ?", 
                    ["56", "63", "72", "54"], 1, "math", "easy", "9 x 7 = 63")
        ]
        self.questions = default_questions
    
    def _load_players(self):
        """플레이어 데이터 로드"""
        try:
            file_path = os.path.join(self.data_dir, 'players.json')
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for name, player_data in data.items():
                        self.players[name] = Player.from_dict(player_data)
        except Exception as e:
            print(f"플레이어 데이터 로드 실패: {e}")
    
    def _save_players(self):
        """플레이어 데이터 저장"""
        try:
            file_path = os.path.join(self.data_dir, 'players.json')
            data = {name: player.to_dict() for name, player in self.players.items()}
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"플레이어 데이터 저장 실패: {e}")
    
    def register_player(self, name):
        """플레이어 등록"""
        if name in self.players:
            self.current_player = self.players[name]
            print(f"기존 플레이어 '{name}' 로그인!")
        else:
            self.current_player = Player(name)
            self.players[name] = self.current_player
            print(f"새 플레이어 '{name}' 등록 완료!")
        return self.current_player
    
    def start_game(self, category='all', difficulty='all', num_questions=5):
        """게임 시작"""
        if not self.current_player:
            raise GameError("플레이어가 등록되지 않았습니다.")
        
        # 문제 필터링
        available_questions = self.questions
        if category != 'all':
            available_questions = [q for q in available_questions if q.category == category]
        if difficulty != 'all':
            available_questions = [q for q in available_questions if q.difficulty == difficulty]
        
        if len(available_questions) < num_questions:
            raise GameError(f"사용 가능한 문제가 부족합니다. (필요: {num_questions}, 사용가능: {len(available_questions)})")
        
        # 랜덤하게 문제 선택
        game_questions = random.sample(available_questions, num_questions)
        
        self.current_game = {
            'questions': game_questions,
            'current_index': 0,
            'score': 0,
            'correct_count': 0,
            'start_time': time.time(),
            'category': category,
            'difficulty': difficulty
        }
        
        print(f"\n🎮 게임 시작! {num_questions}문제를 풀어보세요.")
        return True
    
    def get_current_question(self):
        """현재 문제 반환"""
        if not self.current_game:
            raise GameError("게임이 시작되지 않았습니다.")
        
        index = self.current_game['current_index']
        if index >= len(self.current_game['questions']):
            return None  # 게임 종료
        
        return self.current_game['questions'][index]
    
    def answer_question(self, answer):
        """문제에 답변"""
        if not self.current_game:
            raise GameError("게임이 시작되지 않았습니다.")
        
        question = self.get_current_question()
        if not question:
            raise GameError("더 이상 문제가 없습니다.")
        
        try:
            answer_index = int(answer) - 1  # 1-based to 0-based
            if not (0 <= answer_index < len(question.options)):
                raise ValueError()
        except ValueError:
            raise GameError(f"유효하지 않은 답변입니다. 1-{len(question.options)} 중에서 선택하세요.")
        
        is_correct = answer_index == question.correct
        if is_correct:
            self.current_game['correct_count'] += 1
            self.current_game['score'] += question.get_points()
        
        self.current_game['current_index'] += 1
        
        return {
            'is_correct': is_correct,
            'correct_answer': question.options[question.correct],
            'explanation': question.explanation,
            'points_earned': question.get_points() if is_correct else 0
        }
    
    def end_game(self):
        """게임 종료 및 결과 저장"""
        if not self.current_game:
            raise GameError("진행 중인 게임이 없습니다.")
        
        game_time = time.time() - self.current_game['start_time']
        
        result = {
            'score': self.current_game['score'],
            'correct': self.current_game['correct_count'],
            'total': len(self.current_game['questions']),
            'time': round(game_time, 1),
            'category': self.current_game['category']
        }
        
        # 플레이어 기록에 추가
        self.current_player.add_game_result(
            result['score'], result['correct'], result['total'], result['category']
        )
        
        # 데이터 저장
        self._save_players()
        
        self.current_game = None
        return result
    
    def get_leaderboard(self, limit=10):
        """리더보드 반환"""
        sorted_players = sorted(
            self.players.values(),
            key=lambda p: p.total_score,
            reverse=True
        )
        return sorted_players[:limit]

print("✅ QuizGame 클래스가 정의되었습니다.")

In [ ]:
# === 게임 인터페이스 함수들 ===

def display_menu():
    """메인 메뉴 표시"""
    print("\n" + "="*50)
    print("🧠 PYTHON 종합 퀴즈 게임 🧠")
    print("="*50)
    print("1. 게임 시작")
    print("2. 내 통계 보기")
    print("3. 리더보드")
    print("4. 게임 종료")
    print("-"*50)

def display_question(question, question_num, total_questions):
    """문제 표시"""
    print(f"\n📝 문제 {question_num}/{total_questions} [{question.category}|{question.difficulty}]")
    print("-"*60)
    print(f"Q: {question.text}")
    print()
    for i, option in enumerate(question.options, 1):
        print(f"  {i}. {option}")
    print()

def get_user_input(prompt, valid_options=None):
    """사용자 입력 받기"""
    while True:
        try:
            user_input = input(prompt).strip()
            if valid_options and user_input not in valid_options:
                print(f"유효한 옵션을 선택하세요: {', '.join(valid_options)}")
                continue
            return user_input
        except KeyboardInterrupt:
            print("\n게임을 종료합니다.")
            return None

def play_quiz_round(game):
    """퀴즈 라운드 진행"""
    total_questions = len(game.current_game['questions'])
    
    while True:
        question = game.get_current_question()
        if not question:
            break  # 게임 종료
        
        current_num = game.current_game['current_index'] + 1
        display_question(question, current_num, total_questions)
        
        # 답변 입력
        valid_answers = [str(i) for i in range(1, len(question.options) + 1)]
        answer = get_user_input("답을 선택하세요 (번호): ", valid_answers)
        
        if answer is None:  # 사용자가 종료
            return False
        
        # 답변 처리
        try:
            result = game.answer_question(answer)
            
            if result['is_correct']:
                print(f"\n✅ 정답! +{result['points_earned']}점")
            else:
                print(f"\n❌ 틀렸습니다. 정답: {result['correct_answer']}")
            
            if result['explanation']:
                print(f"💡 설명: {result['explanation']}")
            
            print(f"현재 점수: {game.current_game['score']}점")
            
            # 다음 문제로 넘어가기 전 잠시 대기
            input("\nEnter를 눌러 계속...")
            
        except GameError as e:
            print(f"게임 오류: {e}")
            return False
    
    return True

def show_game_result(result):
    """게임 결과 표시"""
    print("\n" + "="*50)
    print("🏆 게임 완료! 🏆")
    print("="*50)
    print(f"최종 점수: {result['score']}점")
    print(f"정답률: {result['correct']}/{result['total']} ({result['correct']/result['total']*100:.1f}%)")
    print(f"소요 시간: {result['time']}초")
    print(f"카테고리: {result['category']}")
    
    # 성과 평가
    accuracy = result['correct'] / result['total'] * 100
    if accuracy >= 90:
        print("\n🌟 완벽합니다! 정말 훌륭해요!")
    elif accuracy >= 70:
        print("\n👍 잘 했어요! 좋은 실력이네요!")
    elif accuracy >= 50:
        print("\n😊 괜찮아요! 조금만 더 공부하면 될 것 같아요!")
    else:
        print("\n💪 다음에는 더 잘할 수 있을 거예요! 화이팅!")

def show_player_stats(player):
    """플레이어 통계 표시"""
    print("\n" + "="*50)
    print(f"📊 {player.name}님의 통계")
    print("="*50)
    print(f"총 점수: {player.total_score}점")
    print(f"게임 수: {player.total_games}회")
    print(f"전체 정답률: {player.get_accuracy():.1f}%")
    print(f"최고 점수: {player.get_best_score()}점")
    
    if player.game_history:
        print("\n📈 최근 게임 기록:")
        for i, game in enumerate(player.game_history[-3:], 1):  # 최근 3게임
            print(f"  {i}. {game['score']}점 ({game['accuracy']:.1f}%) - {game['category']}")

def show_leaderboard(game):
    """리더보드 표시"""
    leaderboard = game.get_leaderboard()
    
    print("\n" + "="*50)
    print("🏆 리더보드 TOP 10")
    print("="*50)
    
    if not leaderboard:
        print("아직 기록이 없습니다.")
        return
    
    for i, player in enumerate(leaderboard, 1):
        print(f"{i:2d}. {player.name:15s} {player.total_score:6d}점 ({player.get_accuracy():5.1f}%)")

print("✅ 게임 인터페이스 함수들이 정의되었습니다.")

In [ ]:
# === 메인 게임 실행 함수 ===

def main_game():
    """메인 게임 루프"""
    print("🎉 Python 종합 퀴즈 게임에 오신 것을 환영합니다!")
    
    # 게임 초기화
    try:
        game = QuizGame()
        print("✅ 게임 시스템이 초기화되었습니다.")
    except Exception as e:
        print(f"❌ 게임 초기화 실패: {e}")
        return
    
    # 플레이어 등록
    try:
        name = get_user_input("\n플레이어 이름을 입력하세요: ")
        if not name:
            return
        
        player = game.register_player(name)
        print(f"안녕하세요, {player.name}님!")
        
    except InvalidPlayerNameError as e:
        print(f"❌ 이름 오류: {e}")
        return
    except Exception as e:
        print(f"❌ 플레이어 등록 실패: {e}")
        return
    
    # 메인 게임 루프
    while True:
        try:
            display_menu()
            choice = get_user_input("메뉴를 선택하세요: ", ['1', '2', '3', '4'])
            
            if choice is None or choice == '4':
                print("🎮 게임을 종료합니다. 안녕히 가세요!")
                break
            
            elif choice == '1':  # 게임 시작
                try:
                    # 게임 설정
                    print("\n게임 설정:")
                    categories = ['all', 'python', 'math', 'general']
                    difficulties = ['all', 'easy', 'medium', 'hard']
                    
                    print("카테고리: 1)전체 2)Python 3)수학 4)일반상식")
                    cat_choice = get_user_input("카테고리를 선택하세요 (기본:전체): ") or '1'
                    category = categories[int(cat_choice)-1] if cat_choice.isdigit() and 1 <= int(cat_choice) <= 4 else 'all'
                    
                    print("난이도: 1)전체 2)쉬움 3)보통 4)어려움")
                    diff_choice = get_user_input("난이도를 선택하세요 (기본:전체): ") or '1'
                    difficulty = difficulties[int(diff_choice)-1] if diff_choice.isdigit() and 1 <= int(diff_choice) <= 4 else 'all'
                    
                    num_questions = get_user_input("문제 수를 입력하세요 (기본:5): ") or '5'
                    num_questions = int(num_questions) if num_questions.isdigit() else 5
                    num_questions = max(1, min(num_questions, 20))  # 1-20 범위로 제한
                    
                    # 게임 시작
                    game.start_game(category, difficulty, num_questions)
                    
                    # 퀴즈 진행
                    if play_quiz_round(game):
                        result = game.end_game()
                        show_game_result(result)
                    else:
                        print("게임이 중단되었습니다.")
                        if game.current_game:
                            game.end_game()
                    
                except GameError as e:
                    print(f"❌ 게임 오류: {e}")
                except Exception as e:
                    print(f"❌ 예상치 못한 오류: {e}")
            
            elif choice == '2':  # 내 통계
                show_player_stats(game.current_player)
            
            elif choice == '3':  # 리더보드
                show_leaderboard(game)
            
            input("\nEnter를 눌러 메뉴로 돌아가기...")
            
        except KeyboardInterrupt:
            print("\n\n게임을 종료합니다.")
            break
        except Exception as e:
            print(f"❌ 시스템 오류: {e}")
            print("게임을 계속 진행합니다...")

print("✅ 메인 게임 함수가 정의되었습니다.")
print("\n🎯 모든 클래스와 함수가 준비되었습니다!")
print("main_game() 함수를 실행하면 게임이 시작됩니다.")

In [ ]:
# === 게임 시작 ===
# 아래 코드를 실행하면 퀴즈 게임이 시작됩니다!

if __name__ == "__main__":
    try:
        main_game()
    except Exception as e:
        print(f"치명적 오류: {e}")
        print("게임을 종료합니다.")

# 대화형 환경에서는 아래와 같이 실행할 수도 있습니다:
# main_game()

## 🎉 프로젝트 완성!

### 🚀 구현된 기능들

1. **완전한 객체지향 설계**
   - Question, Player, QuizGame 클래스
   - 사용자 정의 예외 처리
   - 데이터 검증 및 캡슐화

2. **파일 입출력 시스템**
   - JSON 형식으로 플레이어 데이터 저장/로드
   - 자동 디렉토리 생성
   - 예외 처리를 통한 안전한 파일 작업

3. **게임 기능들**
   - 카테고리별/난이도별 문제 필터링
   - 점수 시스템 및 통계 관리
   - 실시간 게임 진행 상태 추적
   - 리더보드 시스템

4. **사용자 인터페이스**
   - 직관적인 메뉴 시스템
   - 입력 검증 및 오류 처리
   - 친화적인 피드백 메시지

### 📚 사용된 Python 개념들

- **변수와 자료형**: 다양한 데이터 타입 활용
- **연산자**: 산술, 비교, 논리 연산자
- **자료구조**: 리스트, 딕셔너리, 튜플 활용
- **제어문**: if문, for/while 반복문
- **함수**: 매개변수, 반환값, 람다 함수
- **객체지향**: 클래스, 상속, 캡슐화
- **모듈과 패키지**: import 시스템
- **파일 입출력**: JSON 데이터 처리
- **예외처리**: try-except, 사용자 정의 예외

### 🎯 학습 성과

이 프로젝트를 통해 Python의 모든 핵심 개념을 실제 프로그램에 적용해보며 종합적인 프로그래밍 실력을 기를 수 있었습니다!

**축하합니다! Python 기초 과정을 모두 완주하셨습니다! 🎉**